In [1]:
import numpy as np
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from time import sleep

In [3]:
nombre = []
oferta = []
fecha = []
salario = []
donde = []
remoto = []
teletrabajo = []
localizacion = []
dia_laborable = []
vacaciones = []
tecnologias = []
otras_habilidades = []
idiomas = []
beneficios = []
textos = []

In [4]:
df1 = pd.read_excel("urls_MANFRED.xlsx")

In [5]:
data = list()
cont = 1
for url in df1["URLS"]:
    
    data_manfred = dict()
    print(url)

    data_manfred["url"] = url
    soup = BeautifulSoup(requests.get(url).text, "html.parser")

    #Nombre
    try:
        data_manfred["nombre"] = soup.find('header', class_='sc-1a753b19-1 edWbWI').find('img') ['alt']
    except:
        data_manfred["nombre"] = np.nan
        
    #Oferta
    try:
        data_manfred["oferta"] = soup.find('h1', class_='sc-f61a956f-0 fsOnFB').text
    except:
        data_manfred["oferta"] = np.nan
    
    #Fecha
    try:
        data_manfred["fecha"] = url.split('-')[-1]
        
    except:
        data_manfred["fecha"] = np.nan
    
    #Salario
    try:
        data_manfred["salario"] = soup.find('a', href='#oferta').text
    except:
        data_manfred["salario"] = np.nan

    #Donde
    donde = soup.find_all('a', href='#donde', class_='sc-6d3685c2-2 jPseDG')
    try:
        
        if donde[0].text == '100%':
            try:
                data_manfred["remoto"] = True
                data_manfred["teletrabajo"] = donde[0].text
                data_manfred["localizacion"] = np.nan
            except:
                data_manfred["remoto"] = np.nan
                data_manfred["teletrabajo"] = np.nan
        else:
            try:
                data_manfred["remoto"] = False
                data_manfred["localizacion"] = donde[0].text
                data_manfred["teletrabajo"] = donde[1].text if len(donde) > 1 else None
                
            except:
                data_manfred["remoto"] = np.nan
                data_manfred["localizacion"] = np.nan
                data_manfred["teletrabajo"] = np.nan
    except:
        data_manfred["remoto"] = np.nan
        data_manfred["localizacion"] = np.nan
        data_manfred["teletrabajo"] = np.nan
            
    #Día laborable
    try:
        h3 = soup.find('h3', class_='sc-c51cd5df-3 iJStYn', string='Día laborable')
        span = h3.find_next_sibling('span', class_='sc-c51cd5df-4 cPGQFm')
        data_manfred["dia_laborable"] = span.text.strip()
    except:
        data_manfred["dia_laborable"] = np.nan
        
    #Vacaciones
    try:
        h3 = soup.find('h3', class_='sc-c51cd5df-3 iJStYn', string='Vacaciones')
        span = h3.find_next_sibling('span', class_='sc-c51cd5df-4 cPGQFm')
        data_manfred["vacaciones"] = span.text.strip()
    except:
        data_manfred["vacaciones"] = np.nan

    
    #Beneficios, idiomas, tecnologias, otras_habilidades
    dicc_tablas = dict()

    for section in soup.find_all("section", class_ = "sc-4b1c4ab8-0 hzElAW"):
        dicc_tablas[section.find("h3").text] = list()
    
        if section.find("h3").text == "Beneficios":
            dicc_tablas[section.find("h3").text].extend([x.text for x in section.find_all("div", class_ = "sc-6083f280-0 gJA-dNI")])
    
        if section.find("h3").text == "Idiomas":
            dicc_tablas[section.find("h3").text].extend([x.text for x in section.find_all("div", class_ = "sc-24e58b35-4 hjxecG")])
        
        for ul in section.find_all("ul"):
            dicc_tablas[section.find("h3").text].extend([x.text for x in ul.find_all("h5")])

    data_manfred.update(dicc_tablas)

    #Textos
    try:
        texto_unido = "\n".join([x.text for x in soup.find_all("section", class_ = "sc-d44be200-0 fqoOyX")])

        data_manfred["textos"] = texto_unido
    except:
        data_manfred["textos"] = np.nan

    data.append(data_manfred)

    sleep(1)
    print(cont)
    cont+=1

https://www.getmanfred.com/ofertas-empleo/7684/superreal-tech-lead-nodejs-abr25
1
https://www.getmanfred.com/ofertas-empleo/7648/sngular-js-fullstack-developer-abr25
2
https://www.getmanfred.com/ofertas-empleo/7546/idealista-ux-product-designer-abr25
3
https://www.getmanfred.com/ofertas-empleo/7511/valar-space-java-backend-developer-mar25
4
https://www.getmanfred.com/ofertas-empleo/7406/mavelpoint-software-engineer-lead-mar25
5
https://www.getmanfred.com/ofertas-empleo/6778/eri-bancaire-senior-java-developer-mar25
6
https://www.getmanfred.com/ofertas-empleo/7054/covermanager-tech-lead-mar25
7
https://www.getmanfred.com/ofertas-empleo/6882/tymit-senior-operations-and-incident-specialist-feb25
8
https://www.getmanfred.com/ofertas-empleo/6503/nexthink-senior-software-engineer-abr25
9
https://www.getmanfred.com/ofertas-empleo/7682/novicap-senior-product-manager-abr25
10
https://www.getmanfred.com/ofertas-empleo/7647/mapfre-cloud-architect-abr25
11
https://www.getmanfred.com/ofertas-empleo/

In [6]:
df = pd.json_normalize(data)
df = df.drop('Guardias', axis=1)
df.to_csv("extraccion_manfred.csv", index = False)

In [16]:
df_final = df.drop('Horario', axis=1)
df_final.to_csv("extraccion_manfred_final.csv", index = False)

In [18]:
df_final

,url,nombre,oferta,fecha,salario,remoto,localizacion,teletrabajo,dia_laborable,vacaciones,Tecnologías,Otras habilidades,Idiomas,textos,Beneficios
0,https://www.getmanfred.com/ofertas-empleo/7684...,SuperReal,Tech Lead (Nodejs),abr25,€50-60K,False,Madrid,20%,Jornada completa,25 días,"[NodeJS, REST, TypeScript, Azure]","[Trabajo en equipo, Pensamiento analítico, Lid...",[Inglés],¿Cómo lo harás?SuperReal tiene el tamaño de un...,NaN
1,https://www.getmanfred.com/ofertas-empleo/7648...,Sngular,JS Fullstack Developer,abr25,€35-42K,True,NaN,100%,Jornada completa,25 días,"[NodeJS, REST, MongoDB, SQL, React, TypeScript...","[Adaptabilidad al cambio, Trabajo en equipo, A...",[Inglés],¿Cómo lo harás?Déjame que te describa la quint...,"[Clases de Idiomas, Posibilidad de elegir equi..."
2,https://www.getmanfred.com/ofertas-empleo/7546...,Idealista,UX Product Designer,abr25,Hasta €55K,False,Madrid,60%,Jornada completa,23 días,[Figma],"[Visión crítica, Capacidad de abstracción, Tra...",[Inglés],"¿Cómo lo harás?Es una organización muy plana, ...","[Clases de Idiomas, Café, fruta y snacks, Ayud..."
3,https://www.getmanfred.com/ofertas-empleo/7511...,Valar Space,Java Backend Developer,mar25,€35-40K,True,NaN,100%,Jornada completa,22 días,"[Java, Spring, Hibernate, Github, API, AWS, My...","[Atención al detalle, Capacidad de dar feedbac...","[Español, Inglés]",¿Cómo lo harás?Tu día empezará revisando Linea...,"[Cheque Guardería, Presupuesto para formación,..."
4,https://www.getmanfred.com/ofertas-empleo/7406...,MavelPoint,Senior Frontend Developer,mar25,Hasta €55K,True,NaN,100%,Jornada completa,23 días,"[NextJS, React, PostgreSQL, Azure, nestJS]","[Aprendizaje Continuo, Capacidad de dar feedba...",NaN,"¿Cómo lo harás?Como decíamos más arriba, nos e...",[Presupuesto para formación]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1274,https://www.getmanfred.com/ofertas-empleo/32/g...,G Games,Senior Developer con Buenas Prácticas,jun21,Hasta €45K,True,NaN,100%,NaN,Ilimitado,[],NaN,NaN,¿Cómo lo harás?¡En equipo! ¡Siempre en equipo!...,NaN
1275,https://www.getmanfred.com/ofertas-empleo/28/z...,Zooplus,Senior Frontend Developer,jun21,€40-50K,False,Madrid,60%,Jornada completa,28 días,NaN,NaN,NaN,¿Cómo lo harás?Los equipos de desarrollo traba...,NaN
1276,https://www.getmanfred.com/ofertas-empleo/23/f...,FounderNest,Senior Backend Developer,jun21,€50-60K,True,NaN,100%,NaN,22 días,NaN,NaN,NaN,¿Cómo lo harás?FounderNest tiene una estructur...,NaN
1277,https://www.getmanfred.com/ofertas-empleo/20/c...,Coches.com,PHP Developer,sept21,€40-45K,True,NaN,100%,Jornada completa,22 días,"[PHP, MySQL, Symfony, Laravel, Elasticsearch]","[Aprendizaje Continuo, Trabajo en equipo, Aten...",NaN,¿Cómo lo harás?Se organizan utilizando metodol...,"[Clases de Idiomas, Presupuesto para formación..."


In [ ]:
###################################################################################################################